In [66]:
import re
import random
from autocorrect import Speller

spell = Speller(lang='en')

    
class Chat1(object):
    def __init__(self, pairs, reflections={}):
        """
        Initialize the chatbot.  Pairs is a list of patterns and responses.  Each
        pattern is a regular expression matching the user's statement or question,
        e.g. r'I like (.*)'.  For each such pattern a list of possible responses
        is given, e.g. ['Why do you like %1', 'Did you ever dislike %1'].  Material
        which is matched by parenthesized sections of the patterns (e.g. .*) is mapped to
        the numbered positions in the responses, e.g. %1.

        :type pairs: list of tuple
        :param pairs: The patterns and responses
        :type reflections: dict
        :param reflections: A mapping between first and second person expressions
        :rtype: None
        """

        self._pairs = [(re.compile(x, re.IGNORECASE), y) for (x, y) in pairs]
        self._reflections = reflections
        self._regex = self._compile_reflections()

    def _compile_reflections(self):
        sorted_refl = sorted(self._reflections, key=len, reverse=True)
        return re.compile(
            r"\b({0})\b".format("|".join(map(re.escape, sorted_refl))), re.IGNORECASE
        )

    def _substitute(self, str):
        """
        Substitute words in the string, according to the specified reflections,
        e.g. "I'm" -> "you are"

        :type str: str
        :param str: The string to be mapped
        :rtype: str
        """

        return self._regex.sub(
            lambda mo: self._reflections[mo.string[mo.start() : mo.end()]], str.lower()
        )

    def _wildcards(self, response, match):
        pos = response.find("%")
        while pos >= 0:
            num = int(response[pos + 1 : pos + 2])
            response = (
                response[:pos]
                + self._substitute(match.group(num))
                + response[pos + 2 :]
            )
            pos = response.find("%")
        return response

    def respond(self, str):
        """
        Generate a response to the user input.

        :type str: str
        :param str: The string to be mapped
        :rtype: str
        """

        # check each pattern
        for (pattern, response) in self._pairs:
            match = pattern.match(str)

            # did the pattern match?
            if match:
                resp = random.choice(response)  # pick a random response
                resp = self._wildcards(resp, match)  # process wildcards

                # fix munged punctuation at the end
                if resp[-2:] == "?.":
                    resp = resp[:-2] + "."
                if resp[-2:] == "??":
                    resp = resp[:-2] + "?"
                return resp


    # Hold a conversation with a chatbot
    def converse(self, quit="quit"):
        user_input = ""
        while user_input != quit:
            user_input = quit
            try:
                user_input = input(">")
                user_input = spell(user_input)
            except EOFError:
                print(user_input)
            if user_input:
                while user_input[-1] in "!.":
                    user_input = user_input[:-1]
                print(self.respond(user_input))

In [ ]:
# from nltk.chat.util import Chat
import nltk

reflections = {
    "am": "are",
    "was": "were",
    "i": "u",
    "i'd": "u'd",
    "i've": "u've",
    "ive": "u've",
    "i'll": "u'll",
    "my": "ur",
    "are": "am",
    "you're": "im",
    "you've": "ive",
    "you'll": "i'll",
    "your": "my",
    "yours": "mine",
    "you": "me",
    "u": "me",
    "ur": "my",
    "urs": "mine",
    "me": "u",
}

pairs = [
    [
        r"i need (.*)",
        ["are you sure you aren't being a little greedy", 
         "would you're mom be proud if you %1"]
    ],
     [
        r"who are you?",
        ["i'm gumby, and i'm here to bother you",
        "whatever you want me to be :)"]
    ],
    [
        r"how are (you  | u)?",
        ["ugh.. small talk", "i'm ok. haha"]
    ],
    [
        r"(.*) sorry (.*)",
        ["are you really sorry you %2","its not hard to not %2",]
    ],
    [
        r"hi|hey|hello",
        ["wassup", 
         "yo yo yo, my name is joe. just kidding, its gumby", 
         "good bye. \n\n just kidding haha"]
    ],
    [
        r"(.*) age?",
        ["about as old as ur mum",]
        
    ],
    [
        r"what (.*) want ?",
        ["i just want to be able to climb v12. its not that hard", "i'm a simple man. dirt, rock, and beer"]
        
    ],
    [ r"i cant (.*)",
        ("wow lol, even my gramma can %1", 
     "try harder, maybe you'll be able to %1 some day... haha")
    ],
    [
        r"where do (u|you) live",
        ['on the walls of yosemite baby', 'nunya. \n NUNYA business \n hahaha']
    ],
    [
        r"(how is|how's|hows) the climbing (in|at) (.*)?",
        ["its a high gravity day in %3. don't go",
         "the beers good in %3 but the scenery is lackin really",
         "temps are too hot right now at %3, come by in the winter",
         "never heard about %3"]
    ],
    [ r"(.*)", 
    [".. grumble grumble", "chalk...wheres my chalk", "im getting the heeber jeebers"]],
    [
        r"quit",
        ["see you later alligator","finally!"]],
]

def chatty():
    print("i'm gumby, go away please my plants need watering\n") #default message at the start    
    chat = Chat1(pairs, reflections)
    chat.converse()
if __name__ == "__main__":
    chatty()
    
# nltk.chat.eliza.eliza_chat()



i'm gumby, go away please my plants need watering

>who aree you
whatever you want me to be :)
>hoaw are you
ugh.. small talk
>what is youre age
about as old as ur mum
>how is teh climbin in yosemite
its a high gravity day in yosemite. don't go
>where do you liv
on the walls of yosemite baby


In [82]:
spell = Speller(lang='en')
print(spell('who aree u'))

who are u
